# Compare Baseline vs Basis Timing Strategies

This notebook compares the performance of different strategies.

In [ ]:
import sys
sys.path.insert(0, '/root/sw1')

from datetime import date
import pandas as pd
import matplotlib.pyplot as plt

from src.data.handler import DataHandler
from src.strategy.baseline_roll import BaselineRollStrategy
from src.strategy.basis_timing import BasisTimingStrategy
from src.backtest.engine import BacktestEngine

## 1. Load Data

In [ ]:
handler = DataHandler.from_processed_data('/root/sw1/processed_data', 'IC')

## 2. Run Baseline Strategy

In [ ]:
baseline_strategy = BaselineRollStrategy(
    contract_chain=handler.contract_chain,
    roll_days_before_expiry=2,
    contract_selection='nearby',
    target_leverage=1.0,
)

baseline_engine = BacktestEngine(
    data_handler=handler,
    strategy=baseline_strategy,
    strategy_name='Baseline Roll',
    benchmark_name='CSI 500',
)

baseline_result = baseline_engine.run(
    start_date=date(2018, 1, 1),
    end_date=date(2024, 12, 31),
    verbose=False
)

print("Baseline Strategy:")
baseline_result.analyzer.get_metrics_dataframe().head(15)

## 3. Run Basis Timing Strategy

In [ ]:
# Need to reload handler as strategy modifies state
handler2 = DataHandler.from_processed_data('/root/sw1/processed_data', 'IC')

timing_strategy = BasisTimingStrategy(
    contract_chain=handler2.contract_chain,
    roll_days_before_expiry=2,
    contract_selection='nearby',
    target_leverage=1.0,
    basis_entry_threshold=-0.02,
    basis_exit_threshold=0.005,
)

timing_engine = BacktestEngine(
    data_handler=handler2,
    strategy=timing_strategy,
    strategy_name='Basis Timing',
    benchmark_name='CSI 500',
)

timing_result = timing_engine.run(
    start_date=date(2018, 1, 1),
    end_date=date(2024, 12, 31),
    verbose=False
)

print("Basis Timing Strategy:")
timing_result.analyzer.get_metrics_dataframe().head(15)

## 4. Compare NAV Curves

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

baseline_result.nav_series.plot(ax=ax, label='Baseline Roll', linewidth=1.5)
timing_result.nav_series.plot(ax=ax, label='Basis Timing', linewidth=1.5)
baseline_result.benchmark_nav.plot(ax=ax, label='CSI 500 Index', linewidth=1.5, alpha=0.7)

ax.set_title('Strategy Comparison')
ax.set_ylabel('NAV')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## 5. Compare Metrics

In [ ]:
baseline_metrics = baseline_result.metrics
timing_metrics = timing_result.metrics

comparison = pd.DataFrame({
    'Metric': ['Ann. Return', 'Ann. Volatility', 'Sharpe', 'Max Drawdown', 'Alpha', 'Info Ratio'],
    'Baseline': [
        f"{baseline_metrics['annualized_return']:.2%}",
        f"{baseline_metrics['annualized_volatility']:.2%}",
        f"{baseline_metrics['sharpe_ratio']:.2f}",
        f"{baseline_metrics['max_drawdown']:.2%}",
        f"{baseline_metrics['alpha']:.2%}",
        f"{baseline_metrics['information_ratio']:.2f}",
    ],
    'Basis Timing': [
        f"{timing_metrics['annualized_return']:.2%}",
        f"{timing_metrics['annualized_volatility']:.2%}",
        f"{timing_metrics['sharpe_ratio']:.2f}",
        f"{timing_metrics['max_drawdown']:.2%}",
        f"{timing_metrics['alpha']:.2%}",
        f"{timing_metrics['information_ratio']:.2f}",
    ],
})

comparison